# Como realizar el analisis de datos en Jupyter 

[PyCon Ar en PyVideo.org](https://pyvideo.org/events/pycon-ar-2011.html)

- Explorar los datos

In [19]:
import pathlib

carpeta_datos = pathlib.Path('./datos')

In [33]:
archivos = [ elemento for elemento in carpeta_datos.glob('*/videos/*.json') ]

In [34]:
len(archivos)

237

In [35]:
{archivo.suffix for archivo in archivos }

{'.json'}

In [31]:
archivos[0].suffix

''

- Importar los datos con pandas

In [36]:
import pandas

In [58]:
def safe_df(archivo):
    try:
        df = pandas.read_json(archivo,typ='series')
    except ValueError as err:
        df = pandas.DataFrame()
        print('Fallo al cargar: ', archivo)
    return df

In [59]:
series = [ safe_df(archivo) for archivo in archivos ]

In [69]:
df = pandas.concat(series,axis=1).T

In [73]:
speakers = df.speakers.explode()

In [82]:
import bqplot.pyplot as plt

pandas.set_option('plotting.backend', 'bqplot.pyplot')

In [83]:
df.speakers.apply(len).sort_values().plot()
plt.show()

In [85]:
df.

0                                                  Plone
1                                   Todo sobre deferreds
2                                DBus con el pie derecho
3                  Plone 4: tiempo de cocción 30 minutos
4                          Robótica con Python y Arduino
                             ...                        
232    Reverse-engineering 170 mil líneas de document...
233                   Keynote: Sharing, by Carol Willing
234    Sobreviviendo al síndrome del segundo sistema ...
235             Testing en Proyectos de Machine Learning
236                 Todo lo que no querés saber de async
Name: title, Length: 237, dtype: object

- Realizar graficos